In [20]:
from active_learning.learner.standard import Learner, get_classifier
from active_learning.weak_supervision.SelfTraining import SelfTraining
import argparse
import numpy as np
import copy
import pandas as pd
import math
import random
from sklearn.metrics import accuracy_score, f1_score
from timeit import default_timer as timer
from typing import List, Dict
from active_learning.config import get_active_config
from active_learning.dataStorage import DataStorage
from active_learning.datasets import load_synthetic
from active_learning.logger import init_logger
from active_learning.merge_weak_supervision_label_strategies.MajorityVoteLabelMergeStrategy import (
    MajorityVoteLabelMergeStrategy,
)
from collections import Counter

from active_learning.weak_supervision import SyntheticLabelingFunctions
from active_learning.weak_supervision.BaseWeakSupervision import BaseWeakSupervision

import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_theme(style="whitegrid")


font_size = 8

tex_fonts = {
    # Use LaTeX to write all text
    # "text.usetex": True,
    "text.usetex": False,
    "font.family": ["Times New Roman", 'sans-serif'],
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": font_size,
    "font.size": font_size,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": font_size,
    "xtick.labelsize": font_size,
    "ytick.labelsize": font_size,
    "xtick.bottom": True,
    "figure.autolayout": True,
}

sns.set_style("whitegrid")
sns.set_context("paper")
plt.rcParams.update(tex_fonts)  # type: ignore


# https://jwalton.info/Embed-Publication-Matplotlib-Latex/
def set_matplotlib_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5 ** 0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim


#width = 505.89
width = 1500


df = pd.read_csv("~/exp_results/run_how_to/exp_results.csv")
df

,n_samples,n_features,n_informative,n_redundant,n_repeated,n_classes,n_clusters_per_class,weights,flip_y,class_sep,...,amount_of_lastly_al_samples,JOB_ID,AL_SAMPLES_WEIGHT,AMOUNT_OF_LFS,DATASET,DATASET_RANDOM_GENERATION_SEED,FRACTION_OF_INITIALLY_LABELLED_SAMPLES,FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES,LF_RANDOM_SEED,MERGE_WS_SAMPLES_STRATEGY
0,303,14,14,0,0,2,?,?,?,?,...,35,75,29,1.215732,HEART,28296,0.374271,0.370642,238068,MajorityVoteLabelMergeStrategy
1,194,49,49,0,0,8,?,?,?,?,...,7,100,6,3.108981,flag,628862,0.633401,0.240146,771955,RandomLabelMergeStrategy
2,306,4,4,0,0,2,?,?,?,?,...,85,94,34,2.993717,HABERMAN,618363,0.307777,0.816386,860396,MajorityVoteLabelMergeStrategy
3,178,14,14,0,0,3,?,?,?,?,...,5,0,52,8.927180,wine,644168,0.598658,0.156019,999891,RandomLabelMergeStrategy
4,306,4,4,0,0,2,?,?,?,?,...,75,29,36,1.038934,HABERMAN,751103,0.226496,0.645173,214836,MajorityVoteLabelMergeStrategy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9334,48842,105,105,0,0,4,?,?,?,?,...,9706,9245,50,4.401298,adult,869522,0.533442,0.852170,721962,SnorkelLabelMergeStrategy
9335,10000,3073,3073,0,0,10,?,?,?,?,...,2994,9110,65,2.502163,cifar10,673335,0.399166,0.999769,280095,MajorityVoteLabelMergeStrategy
9336,195,23,23,0,0,2,?,?,?,?,...,8,9103,33,4.402870,parkinsons,616999,0.229822,0.102335,907849,MajorityVoteLabelMergeStrategy
9337,1000,25,25,0,0,2,?,?,?,?,...,214,9107,39,1.402683,GERMAN,494724,0.485892,0.836036,859891,MajorityVoteLabelMergeStrategy


In [21]:
df.keys()#

Index(['n_samples', 'n_features', 'n_informative', 'n_redundant', 'n_repeated',
       'n_classes', 'n_clusters_per_class', 'weights', 'flip_y', 'class_sep',
       'hypercube', 'scale', 'random_state', 'ABSTAIN_THRESHOLDS',
       'LF_CLASSIFIERS', 'AMOUNT_OF_LF_FEATURESSS', 'f1_initial',
       'acc_initial', 'f1_ws', 'acc_ws',
       'acc_ws_and_al_UncertaintyMaxMargin_no_ws',
       'f1_ws_and_al_UncertaintyMaxMargin_no_ws',
       'acc_al_and_al_UncertaintyMaxMargin_no_ws',
       'f1_al_and_al_UncertaintyMaxMargin_no_ws',
       'acc_ws_and_al_UncertaintyMaxMargin_with_ws',
       'f1_ws_and_al_UncertaintyMaxMargin_with_ws',
       'acc_al_and_al_UncertaintyMaxMargin_with_ws',
       'f1_al_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random',
       'f1_ws_and_al_Random', 'acc_al_and_al_Random', 'f1_al_and_al_Random',
       'acc_ws_and_al_CoveredByLeastAmountOfLf',
       'f1_ws_and_al_CoveredByLeastAmountOfLf',
       'acc_al_and_al_CoveredByLeastAmountOfLf',
       'f

In [22]:
def hist_plot(df: pd.DataFrame, COMPARE_KEYS:List[str], TITLE:str=None, OUTPUT_PATH: str=None, LABEL:str="AUC-F1 (\\%)"):
    fig = plt.figure(figsize=set_matplotlib_size(width, fraction=0.5))

    # fig = plt.gcf()  # type: ignore
    # fig.set_size_inches(18.5, 10.5)

    selection_list = []
    for key in COMPARE_KEYS:
        selection_list.append((df[key], key))
    #for strategy in df["strategy"].unique():
    #    selection_list.append((df[df["strategy"] == strategy]["f1_auc"], strategy))

    for selection, label in sorted(selection_list, key=lambda tup: tup[1]):
        #  ax = sns.distplot(selection, label=label, **kwargs)
        #  label = "{}-{:>3}: {:.4g}% ".format(label[0], label[1], selection.mean() * 100)
        label = "{:<10}{:>5.4g}% ".format(label + ":", selection.mean() * 100)
        ax = sns.kdeplot(selection, label=label)
        # ax = sns.distplot(selection, label=label, kde=True)
        #ax.set_xlim(0, 1)
        ax.set_xlim(0.5, 0.6)
        #  ax.set_xlim(0.75, 0.8)
        #  ax.set_xlim(80, 875)

        mean = selection.mean()
        if selection.count() == 0:
            low = high = mean
        else:
            low = selection.mean() - 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
            high = selection.mean() + 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        print(label)
        ax.axvline(mean, color=plt.gca().lines[-1].get_color())  # type: ignore
        ax.axvspan(low, high, alpha=0.2, color=plt.gca().lines[-1].get_color())  # type: ignore

        ax.set_xticks([x for x in ax.get_xticks()]) # stupid "workaround" for https://github.com/matplotlib/matplotlib/issues/18848
        ax.set_xticklabels(["{:.0%}\\%".format(x) for x in ax.get_xticks()])
    ax.set_title(TITLE)  # type: ignore
    # ax.set_ylabel("")
    ax.set_xlabel(LABEL)  # type: ignore
    ax.legend(  # type: ignore
        loc="center left"
    )  # , bbox_to_anchor=(0.0, 1.01), borderaxespad=0, frameon=False, ncol=len(selection))  # type: ignore
    # ax.xa

def comparison_table(df: pd.DataFrame, COMPARE_KEYS:List[str], TITLE:str=""):
    results = []
    for key in COMPARE_KEYS:
        selection = df[key]

        mean = selection.mean()
        error_area = 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        results.append([key, mean, error_area])
    print(TITLE)
    print("{:>60} {:>6} {:>4}".format("", "Mean", ""))
    for result in results:
        print("{:<50} {:>6.2%} ±{:>4.2%}".format(result[0], float(result[1]), float(result[2])))

def answer_questions(df: pd.DataFrame, COMPARE_KEYS:List[str], metric):
    results = {}
    for key in COMPARE_KEYS:
        selection = df[key]

        mean = selection.mean()
        error_area = 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        results[key] = (mean, error_area)
    
    better_than_initial = [k for k,v in results.items() if v[0] > results[metric + "_initial"][0]]
    better_than_initial_with_ws = [k for k in better_than_initial if "ws_and_al" in k]
    print("Which are better than initial?", better_than_initial)
    print("Which are better than initial, with WS?", better_than_initial_with_ws)

    al_ws_al_better_than_al_al = []

    for k in results.keys():
        if k.startswith(metric + "_ws_and_al"):
            if results[k][0] > results[k.replace("_ws_and_al", "_al_and_al")][0]:
                al_ws_al_better_than_al_al.append(k)

    print("Where is al+ws+al better than al+al?", al_ws_al_better_than_al_al)
    
    #print("Where is the mean improvement within the 95% error range?")
    
    
# 01 general improvement
for metric in ['acc', 'f1']:
    COMPARE_KEYS = [metric+"_initial", metric +"_ws"]
    for AL_SAMPLING_STRATEGY in ["UncertaintyMaxMargin_no_ws",
            "UncertaintyMaxMargin_with_ws",
            "Random",
            "CoveredByLeastAmountOfLf",
            "ClassificationIsMostWrong",
            "GreatestDisagreement"]:
            COMPARE_KEYS.append(metric + '_ws_and_al_'+AL_SAMPLING_STRATEGY)
            COMPARE_KEYS.append(metric + '_al_and_al_'+AL_SAMPLING_STRATEGY)
    comparison_table(df, COMPARE_KEYS=COMPARE_KEYS, TITLE="General Improvement after AL + WS " + metric)
    answer_questions(df, COMPARE_KEYS=COMPARE_KEYS, metric=metric)

General Improvement after AL + WS acc
                                                               Mean     
acc_initial                                        58.06% ±0.45%
acc_ws                                             56.00% ±0.44%
acc_ws_and_al_UncertaintyMaxMargin_no_ws           57.71% ±0.45%
acc_al_and_al_UncertaintyMaxMargin_no_ws           58.61% ±0.46%
acc_ws_and_al_UncertaintyMaxMargin_with_ws         57.71% ±0.45%
acc_al_and_al_UncertaintyMaxMargin_with_ws         58.61% ±0.46%
acc_ws_and_al_Random                               57.59% ±0.44%
acc_al_and_al_Random                               58.21% ±0.46%
acc_ws_and_al_CoveredByLeastAmountOfLf             57.17% ±0.45%
acc_al_and_al_CoveredByLeastAmountOfLf             58.16% ±0.46%
acc_ws_and_al_ClassificationIsMostWrong            56.98% ±0.44%
acc_al_and_al_ClassificationIsMostWrong            57.50% ±0.46%
acc_ws_and_al_GreatestDisagreement                 57.13% ±0.44%
acc_al_and_al_GreatestDisagreement          

In [23]:


# group by stuff
for group_by in ['DATASET', 'MERGE_WS_SAMPLES_STRATEGY']:
    print()
    print("#"*80)
    print(group_by)
    print("#"*80)
    for group_by_value in df[group_by].unique():
        print()
        print(group_by_value, ": ", len(df[df[group_by] == group_by_value]))
        for metric in ['acc', 'f1']:
            COMPARE_KEYS = [metric+"_initial", metric +"_ws"]
            for AL_SAMPLING_STRATEGY in ["UncertaintyMaxMargin_no_ws",
                    "UncertaintyMaxMargin_with_ws",
                    "Random",
                    "CoveredByLeastAmountOfLf",
                    "ClassificationIsMostWrong",
                    "GreatestDisagreement"]:
                    COMPARE_KEYS.append(metric + '_ws_and_al_'+AL_SAMPLING_STRATEGY)
                    COMPARE_KEYS.append(metric + '_al_and_al_'+AL_SAMPLING_STRATEGY)
            #comparison_table(df, COMPARE_KEYS=COMPARE_KEYS, TITLE="General Improvement after AL + WS " + metric)
            answer_questions(df[df[group_by] == group_by_value], COMPARE_KEYS=COMPARE_KEYS, metric=metric)


################################################################################
DATASET
################################################################################

HEART :  315
Which are better than initial? ['acc_ws', 'acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement']
Which are better than initial, with WS? ['acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement']
Where is al+ws+al better than al+al? ['acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_CoveredByLeastAmountOfLf', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement']
Which are better than initial? ['f1_ws', 'f1_ws_and_al_UncertaintyM

In [24]:
#andersherum an die Sache gehen : im großen df schauen wo al_ws_al better than al+al -> und dann schauen wie viel prozent davon jeweils auf die einzelnen werte von datasets, … verteilt sind!!
df1 = pd.DataFrame({'city':['A','B','C'],
                    'keep1':['D', 'E', 'F'],
                    'f1_ws_and_al_U':[1,2,3],
                    'f1_al_and_al_U':[4,5,6],
                    'f1_ws_and_al_R':[7,8,9],
                    'f1_al_and_al_R':[10,11,12],
                    'f1_ws_and_al_X':[13,14,15],
                    'f1_al_and_al_X':[16,17,18],

                    'acc_ws_and_al_U':[19,20,21],
                    'acc_al_and_al_U':[22,23,24],
                    'acc_ws_and_al_R':[25,26,27],
                    'acc_al_and_al_R':[27,28,29],
                    'acc_ws_and_al_X':[30,31,32],
                    'acc_al_and_al_X':[33,34,35],
})
print(df1)




df_new = pd.DataFrame()
al_sampling_strategies = ["U", "R", "X"]
to_split_metrics = (
    ["f1_ws_and_al_" + k for k in al_sampling_strategies]
    + ["f1_al_and_al_" + k for k in al_sampling_strategies]
    + ["acc_ws_and_al_" + k for k in al_sampling_strategies]
    + ["acc_al_and_al_" + k for k in al_sampling_strategies]
)
print(to_split_metrics)

for _, row in df1.iterrows():
    for al_sampling_strategy in al_sampling_strategies:
        # remove all other metrics
        row2 = row.drop(
            [m for m in to_split_metrics if not m.endswith(al_sampling_strategy)]
        )
        row2 = row2.rename(
            {
                "f1_ws_and_al_" + al_sampling_strategy: "f1_al_and_ws_and_al",
                "acc_ws_and_al_" + al_sampling_strategy: "acc_al_and_ws_and_al",
                "f1_al_and_al_" + al_sampling_strategy: "f1_al_and_al",
                "acc_al_and_al_" + al_sampling_strategy: "acc_al_and_al",
            }
        )
        df_new = df_new.append(row2)

df_new

  city keep1  f1_ws_and_al_U  f1_al_and_al_U  f1_ws_and_al_R  f1_al_and_al_R  \
0    A     D               1               4               7              10   
1    B     E               2               5               8              11   
2    C     F               3               6               9              12   

   f1_ws_and_al_X  f1_al_and_al_X  acc_ws_and_al_U  acc_al_and_al_U  \
0              13              16               19               22   
1              14              17               20               23   
2              15              18               21               24   

   acc_ws_and_al_R  acc_al_and_al_R  acc_ws_and_al_X  acc_al_and_al_X  
0               25               27               30               33  
1               26               28               31               34  
2               27               29               32               35  
['f1_ws_and_al_U', 'f1_ws_and_al_R', 'f1_ws_and_al_X', 'f1_al_and_al_U', 'f1_al_and_al_R', 'f1_al_and_al_X

,acc_al_and_al,acc_al_and_ws_and_al,city,f1_al_and_al,f1_al_and_ws_and_al,keep1
0,22.0,19.0,A,4.0,1.0,D
0,27.0,25.0,A,10.0,7.0,D
0,33.0,30.0,A,16.0,13.0,D
1,23.0,20.0,B,5.0,2.0,E
1,28.0,26.0,B,11.0,8.0,E
1,34.0,31.0,B,17.0,14.0,E
2,24.0,21.0,C,6.0,3.0,F
2,29.0,27.0,C,12.0,9.0,F
2,35.0,32.0,C,18.0,15.0,F


In [25]:
CREATE_CSV = True
if CREATE_CSV:
    # unravel al sampling strat metrics
    al_sampling_strategies = [
        "UncertaintyMaxMargin_no_ws",
        "UncertaintyMaxMargin_with_ws",
        "Random",
        "CoveredByLeastAmountOfLf",
        "ClassificationIsMostWrong",
        "GreatestDisagreement",
    ]

    df_new = pd.DataFrame()
    to_split_metrics = (
        ["f1_ws_and_al_" + k for k in al_sampling_strategies]
        + ["f1_al_and_al_" + k for k in al_sampling_strategies]
        + ["acc_ws_and_al_" + k for k in al_sampling_strategies]
        + ["acc_al_and_al_" + k for k in al_sampling_strategies]
    )
    print(to_split_metrics)

    for _, row in df.iterrows():
        for al_sampling_strategy in al_sampling_strategies:
            # remove all other metrics
            row2 = row.drop(
                [m for m in to_split_metrics if not m.endswith(al_sampling_strategy)]
            )
            row2 = row2.rename(
                {
                    "f1_ws_and_al_" + al_sampling_strategy: "f1_al_and_ws_and_al",
                    "acc_ws_and_al_" + al_sampling_strategy: "acc_al_and_ws_and_al",
                    "f1_al_and_al_" + al_sampling_strategy: "f1_al_and_al",
                    "acc_al_and_al_" + al_sampling_strategy: "acc_al_and_al",
                }
            )
            row2['al_sampling_strategy'] = al_sampling_strategy
            df_new = df_new.append(row2)

    df_new

['f1_ws_and_al_UncertaintyMaxMargin_no_ws', 'f1_ws_and_al_UncertaintyMaxMargin_with_ws', 'f1_ws_and_al_Random', 'f1_ws_and_al_CoveredByLeastAmountOfLf', 'f1_ws_and_al_ClassificationIsMostWrong', 'f1_ws_and_al_GreatestDisagreement', 'f1_al_and_al_UncertaintyMaxMargin_no_ws', 'f1_al_and_al_UncertaintyMaxMargin_with_ws', 'f1_al_and_al_Random', 'f1_al_and_al_CoveredByLeastAmountOfLf', 'f1_al_and_al_ClassificationIsMostWrong', 'f1_al_and_al_GreatestDisagreement', 'acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_CoveredByLeastAmountOfLf', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement', 'acc_al_and_al_UncertaintyMaxMargin_no_ws', 'acc_al_and_al_UncertaintyMaxMargin_with_ws', 'acc_al_and_al_Random', 'acc_al_and_al_CoveredByLeastAmountOfLf', 'acc_al_and_al_ClassificationIsMostWrong', 'acc_al_and_al_GreatestDisagreement']


In [26]:
if CREATE_CSV:
    df_new.to_csv("how_to.csv", index=False)

In [27]:

df = pd.read_csv("how_to.csv")

print(df.keys())
print("Total: ", len(df))

no_hist_keys = [
    "ABSTAIN_THRESHOLDS",
    "AL_SAMPLES_WEIGTHS",
    "DATASET_RANDOM_GENERATION_SEED",
    "JOB_ID",
    "LF_CLASSIFIERS",
    "LF_RANDOM_SEED",
    "random_state",
    "AMOUNT_OF_LF_FEATURESSS",
]


hist_keys_that_are_lists =  [
    "ABSTAIN_THRESHOLDS",
    "AL_SAMPLES_WEIGTHS",
    "LF_CLASSIFIERS",
    "AMOUNT_OF_LF_FEATURESSS",
]

# 1. check the value ranges -> does it make sense to go one_hot_encoding? for LF_CLASSIFIERS definitely
# 2. for the rest not so much -,-


title="all"
for key in df:
    if key.startswith("acc_") or key.startswith("f1_") or key in no_hist_keys:
        continue
    print(key)
    sns.histplot(df, y=key).set(title=title)
    plt.savefig('plots/' + title+ '_' + key + '.jpg', dpi=300, bbox_inches="tight")
    plt.savefig('plots/' + title+ '_' + key + '.pdf',dpi=300, format="pdf", bbox_inches="tight")
    plt.clf()


def analyse_value_distributions(df, df_a, df_not_a, title):
    for key in df_a:
        if key.startswith("acc_") or key.startswith("f1_") or key in no_hist_keys:
            continue
        print(key)

        f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, sharex=True)
        ax1.set_title(title)
        sns.histplot(df_a, y=key, ax=ax1)
        sns.histplot(df_not_a, y=key, ax=ax2)
        plt.savefig("plots/" + title + "_" + key + ".jpg", dpi=300, bbox_inches="tight")
        plt.savefig(
            "plots/" + title + "_" + key + ".pdf",
            dpi=300,
            format="pdf",
            bbox_inches="tight",
        )
        plt.clf()
        #acc per lf als stärke der lf

for metric in ["acc", "f1"]:
    print(metric)
        
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_initial"] > df[metric + "_al_and_al"])
            & (df[metric + "_initial"] > df[metric + "_al_and_ws_and_al"])],
        df.loc[(df[metric + "_initial"] <= df[metric + "_al_and_al"])
            & (df[metric + "_initial"] <= df[metric + "_al_and_ws_and_al"])],
        metric + " Initial was better than everything else",
    )
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_ws"] > df[metric + "_initial"])],
        df.loc[(df[metric + "_ws"] >= df[metric + "_initial"])],
         metric + "Only WS > Initial",
    )
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_al_and_ws_and_al"] > df[metric + "_ws"])],
        df.loc[(df[metric + "_al_and_ws_and_al"] <= df[metric + "_ws"])],
         metric + " AL and WS and AL > Only WS",
    )
    analyse_value_distributions(
        df,
        df.loc[ (df[metric + "_al_and_al"] > df[metric + "_initial"])],
        df.loc[ (df[metric + "_al_and_al"] <= df[metric + "_initial"])],
         metric + " AL and AL > AL",
    )
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_al_and_ws_and_al"] < df[metric + "_ws"])],
        df.loc[(df[metric + "_al_and_ws_and_al"] >= df[metric + "_ws"])],
        metric + " AL and WS and AL < Only WS",
    )

Index(['ABSTAIN_THRESHOLDS', 'AL_SAMPLES_WEIGHT', 'AMOUNT_OF_LFS',
       'AMOUNT_OF_LF_FEATURESSS', 'DATASET', 'DATASET_RANDOM_GENERATION_SEED',
       'FRACTION_OF_INITIALLY_LABELLED_SAMPLES',
       'FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES', 'JOB_ID', 'LF_CLASSIFIERS',
       'LF_RANDOM_SEED', 'MERGE_WS_SAMPLES_STRATEGY', 'acc_al_and_al',
       'acc_al_and_ws_and_al', 'acc_initial', 'acc_ws', 'al_sampling_strategy',
       'amount_of_initial_al_samples', 'amount_of_lastly_al_samples',
       'class_sep', 'f1_al_and_al', 'f1_al_and_ws_and_al', 'f1_initial',
       'f1_ws', 'flip_y', 'hypercube', 'n_classes', 'n_clusters_per_class',
       'n_features', 'n_informative', 'n_redundant', 'n_repeated', 'n_samples',
       'random_state', 'scale', 'weights'],
      dtype='object')
Total:  56034
AL_SAMPLES_WEIGHT
AMOUNT_OF_LFS
DATASET
FRACTION_OF_INITIALLY_LABELLED_SAMPLES
FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES
MERGE_WS_SAMPLES_STRATEGY
al_sampling_strategy
amount_of_initial_al_samples
amou

<ipython-input-27-05b32b7eac77>:35: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.savefig('plots/' + title+ '_' + key + '.jpg', dpi=300, bbox_inches="tight")
<ipython-input-27-05b32b7eac77>:36: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.savefig('plots/' + title+ '_' + key + '.pdf',dpi=300, format="pdf", bbox_inches="tight")
<ipython-input-27-05b32b7eac77>:36: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.savefig('plots/' + title+ '_' + key + '.pdf',dpi=300, format="pdf", bbox_inches="tight")


acc
AL_SAMPLES_WEIGHT
AMOUNT_OF_LFS
DATASET
FRACTION_OF_INITIALLY_LABELLED_SAMPLES
FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES
MERGE_WS_SAMPLES_STRATEGY
al_sampling_strategy
amount_of_initial_al_samples
amount_of_lastly_al_samples
class_sep
flip_y
hypercube
n_classes
n_clusters_per_class
n_features
n_informative
n_redundant
n_repeated
n_samples
scale


<ipython-input-27-05b32b7eac77>:46: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, sharex=True)


weights


...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>